In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
#what can be done next?
#1. exploit more on loss function
#2. make unet deeper/wider?

In [ ]:
#!pip install albumentations > /dev/null

In [ ]:
from __future__ import print_function
import os,sys,random,cv2,re
import tensorflow as tf
import tifffile
from tensorflow import keras
#from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
from tensorflow import reduce_sum
from tensorflow.keras.backend import pow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, UpSampling2D, Concatenate, Add, Flatten
from tensorflow.keras.losses import binary_crossentropy


from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob
import skimage.io as io
import skimage.transform as trans
import matplotlib.pyplot as plt
import shutil
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import LearningRateScheduler

from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

image_size = 512
batch_size = 2
h,w = 512,512

> ***** Train Test Split

In [ ]:
#test-train split
imageData = os.listdir("../input/images")
imageLabel = os.listdir("../input/labels")
#ensure image and labels correspond
imageData.sort()
imageLabel.sort()
trainImage,validImage,trainLable,validLabel = train_test_split(imageData,imageLabel,test_size=0.2,shuffle=True,random_state=2)
def addPrefix(x):
    pre = x.split("-")[1][0:3]
    if(pre == "vol"):
        return "../input/images/" + x
    elif(pre == "lab"):
        return "../input/labels/" + x
    else:
        return "Invalid input"
tmp = [trainImage,validImage,trainLable,validLabel]

trainImage = list(map(addPrefix,trainImage))
validImage = list(map(addPrefix,validImage))
trainLable = list(map(addPrefix,trainLable))
validLabel = list(map(addPrefix,validLabel))

Image Augumentation

Elastic Transform

In [ ]:
def elastic_transform(image, alpha, sigma, alpha_affine, random_state=None):
    """Elastic deformation of images as described in [Simard2003]_ (with modifications).
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
         Convolutional Neural Networks applied to Visual Document Analysis", in
         Proc. of the International Conference on Document Analysis and
         Recognition, 2003.

     Based on https://gist.github.com/erniejunior/601cdf56d2b424757de5
    """
    if random_state is None:
        random_state = np.random.RandomState(None)
    shape = image.shape
    shape_size = shape[:2]
    
    # Random affine
    center_square = np.float32(shape_size) // 2
    square_size = min(shape_size) // 3
    pts1 = np.float32([center_square + square_size, [center_square[0]+square_size, center_square[1]-square_size], center_square - square_size])
    pts2 = pts1 + random_state.uniform(-alpha_affine, alpha_affine, size=pts1.shape).astype(np.float32)
    M = cv2.getAffineTransform(pts1, pts2)
    image = cv2.warpAffine(image, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)

    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    dz = np.zeros_like(dx)

    x, y, z = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]), np.arange(shape[2]))
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1)), np.reshape(z, (-1, 1))

    return map_coordinates(image, indices, order=1, mode='reflect').reshape(shape)

In [ ]:
# Define function to draw a grid
def draw_grid(im, grid_size):
    # Draw grid lines
    for i in range(0, im.shape[1], grid_size):
        cv2.line(im, (i, 0), (i, im.shape[0]), color=(255,))
    for j in range(0, im.shape[0], grid_size):
        cv2.line(im, (0, j), (im.shape[1], j), color=(255,))

# Load images
im = cv2.imread("../input/images/train-volume00.jpg", -1)
im_mask = cv2.imread("../input/labels/train-labels00.jpg", -1)
# Draw grid lines
draw_grid(im, 50)
draw_grid(im_mask, 50)

# Merge images into separete channels (shape will be (cols, rols, 2))

im_merge = np.concatenate((im[...,None], im_mask[...,None]), axis=2)

In [ ]:
def elasticTrans(img, msk, batchSize = 2):
    length = img.shape[1]
    for i in range(img.shape[0]):
        im_merge = np.concatenate((img[i], msk[i]), axis=2)
        im_merge_t = elastic_transform(im_merge, im_merge.shape[1] * 2, \
                                       im_merge.shape[1] * 0.08,im_merge.shape[1] * 0.08)
        img[i] = im_merge_t[..., 0].reshape(length, length, 1)
        #plt.imshow(img[i].reshape(256,256), 'gray')
        #plt.show()
        msk[i] = im_merge_t[..., 1].reshape(length, length, 1)
    return img, msk

In [ ]:
# First sample...

%matplotlib inline

# Apply transformation on image
im_merge_t = elastic_transform(im_merge, im_merge.shape[1] * 2, im_merge.shape[1] * 0.08, im_merge.shape[1] * 0.08)

# Split image and mask
im_t = im_merge_t[...,0]
im_mask_t = im_merge_t[...,1]

# Display result
plt.figure(figsize = (16,14))
plt.imshow(np.c_[np.r_[im, im_mask], np.r_[im_t, im_mask_t]], cmap='gray')

In [ ]:
#1. 数据增强 Data augmentation的参数

def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    stacked = []
    for i,item in enumerate(npyfile):
        img = item[:,:,0].astype('float32')
        img = np.expand_dims(img,axis=0)
        if(len(stacked)==0):
            stacked = img
        else:
            stacked = np.vstack((stacked,img))
    tifffile.imsave(save_path + '/allPredict.tif', stacked)
        #tifffile.imsave(, tiff_list)
        #io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)

def adjustData(img,mask,flag_multi_class,num_class):
    if(np.max(img) > 1):
        #divided by 255 to convert to probability image
        img = img / 255
        mask = mask /255
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
    return (img,mask)

data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='reflect')

def trainGenerator(batch_size,trainImage,trainLable,aug_dict,image_color_mode = "grayscale",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (512,512),seed = 1):

    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    
    image_generator = image_datagen.flow(
        x = np.array([cv2.imread(ele,0).reshape((512,512,1)) for ele in trainImage]),
        batch_size = batch_size,
        shuffle = True,
        seed = seed,
        save_to_dir = save_to_dir 
    )
    mask_generator = mask_datagen.flow(
        x = np.array([cv2.imread(ele,0).reshape((512,512,1)) for ele in trainLable]),
        batch_size = batch_size,
        shuffle = True,
        seed = seed,
        save_to_dir = save_to_dir 
    )

    train_generator = zip(image_generator, mask_generator)
#     counter = 0
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
#         if(counter%10 == 0):
#             img,mask = elasticTrans(img,mask)
#         counter += 1
        yield (img,mask)
        
myGene = trainGenerator(batch_size,trainImage,trainLable,data_gen_args,save_to_dir = None)
myValid = trainGenerator(batch_size,validImage,validLabel,data_gen_args,save_to_dir = None)

1. **Visualize data augumentation result**

In [ ]:
a = myGene.__next__()
k = 1
plt.figure(figsize=(15,15))
for i in range(2):
    for j in range(2):
        plt.subplot(2,2,k)
        plt.imshow(a[i][j].reshape(512,512),'gray')
        k+=1
plt.show()

Loss Functions

In [ ]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred, smooth = 1):
    return 1- dice_coef(y_true, y_pred, smooth)

def bce_dice_loss(y_true, y_pred, weight = 1, smooth = 1):
    '''
    bce + w*dice loss
    '''
    return binary_crossentropy(y_true, y_pred) + weight*dice_loss(y_true, y_pred)

2.Define Unet Model 

In [ ]:
# def down_block(x,filters,kernel_size=(3,3),padding='same',strides=1):
#     c=keras.layers.Conv2D(filters,kernel_size,padding=padding,strides=strides,activation='relu',kernel_initializer = 'he_normal')(x)
#     c=keras.layers.Conv2D(filters,kernel_size,padding=padding,strides=strides,activation='relu',kernel_initializer = 'he_normal')(c)
#     p=keras.layers.MaxPool2D((2,2),(2,2))(c)
#     return c,p
# def up_block(x,skip,filters,kernel_size=(3,3),padding='same',strides=1):
#     us=keras.layers.UpSampling2D((2,2))(x)
#     concat=keras.layers.concatenate([us,skip],axis=3)
#     c=keras.layers.Conv2D(filters,kernel_size,padding=padding,strides=strides,activation='relu',kernel_initializer = 'he_normal')(concat)
#     c=keras.layers.Conv2D(filters,kernel_size,padding=padding,strides=strides,activation='relu',kernel_initializer = 'he_normal')(c)
#     return c
# def bottleNeck(x,filters,kernel_size=(3,3),padding='same',strides=1):
#     c=keras.layers.Conv2D(filters,kernel_size,padding=padding,strides=strides,activation='relu',kernel_initializer = 'he_normal')(x)
#     c=keras.layers.Conv2D(filters,kernel_size,padding=padding,strides=strides,activation='relu',kernel_initializer = 'he_normal')(c)
#     return c
# def Unet():
#     f = [64,128,256,512,1024]#feature maps, number of filters
#     inputs=keras.layers.Input((image_size,image_size,1))#input to the model
#     p0 = inputs
#     c1,p1 = down_block(p0,f[0])#1->64
#     c2,p2 = down_block(p1,f[1])#64->128
#     c3,p3 = down_block(p2,f[2])#128->256
#     c4,p4 = down_block(p3,f[3])#256->512
#     drop1 = keras.layers.Dropout(0.5)(p4)
    
#     bn = bottleNeck(drop1,f[4])#512->1024
#     drop2 = keras.layers.Dropout(0.5)(bn)

#     u1 = up_block(drop2,c4,f[3])#1024->512
#     u2 = up_block(u1,c3,f[2])#512->256
#     u3 = up_block(u2,c2,f[1])#256->128
#     u4 = up_block(u3,c1,f[0])#128->64
    
#     outputs_up = keras.layers.Conv2D(2,(3,3),padding="same",activation="relu",kernel_initializer = 'he_normal')(u4)#64->2
#     outputs = keras.layers.Conv2D(1,(1,1),padding="same",activation="sigmoid")(outputs_up)#2->1
#     print("output",outputs.shape)
#     model = keras.models.Model(inputs,outputs)
#     return model







def unet(pretrained_weights = None,input_size = (512, 512, 1)):
    inputs = keras.layers.Input(input_size)
    conv1 = keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = keras.layers.Dropout(0.5)(conv4)
    pool4 = keras.layers.MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = keras.layers.Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = keras.layers.Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = keras.layers.Dropout(0.5)(conv5)

    up6 = keras.layers.Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = keras.layers.concatenate([drop4,up6], axis = 3)
    conv6 = keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = keras.layers.Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = keras.layers.concatenate([conv3,up7], axis = 3)
    conv7 = keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = keras.layers.Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = keras.layers.concatenate([conv2,up8], axis = 3)
    conv8 = keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = keras.layers.Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = keras.layers.concatenate([conv1,up9], axis = 3)
    conv9 = keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = keras.layers.Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = keras.layers.Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = keras.models.Model(inputs,conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = [dice_loss], metrics = ['accuracy', dice_coef])
    #model.compile(optimizer = Adam(lr = 1e-4), loss = focal_loss(gamma=2,alpha=0.6), metrics = ['accuracy', dice_coef])
    #model.compile(optimizer = Adam(lr = 1e-4), loss = focal_dice_loss, metrics = ['accuracy', dice_coef])
    
    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

ResUnet

In [ ]:
# def bn_act(x, act=True):
#     'batch normalization layer with an optinal activation layer'
#     x = tf.keras.layers.BatchNormalization()(x)
#     if act == True:
#         x = tf.keras.layers.Activation('relu')(x)
#     return x
# def conv_block(x, filters, kernel_size=3, padding='same', strides=1):#BN + RELU + CONV
#     'convolutional layer which always uses the batch normalization layer'
#     conv = bn_act(x)
#     conv = Conv2D(filters, kernel_size, padding=padding, strides=strides)(conv)
#     return conv
# def stem(x, filters, kernel_size=3, padding='same', strides=1):
#     conv = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
#     conv = conv_block(conv, filters, kernel_size, padding, strides)
#     shortcut = Conv2D(filters, kernel_size=1, padding=padding, strides=strides)(x)
#     shortcut = bn_act(shortcut, act=False)
#     output = Add()([conv, shortcut])
#     return output
# def residual_block(x, filters, kernel_size=3, padding='same', strides=1):
#     res = conv_block(x, filters, 3, padding, strides)
#     res = conv_block(res, filters, 3, padding, 1)
#     shortcut = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
#     shortcut = bn_act(shortcut, act=False)
#     output = Add()([shortcut, res])
#     return output
# def upsample_concat_block(x, xskip):
#     u = UpSampling2D((2,2))(x)
#     c = Concatenate()([u, xskip] )
#     return c
# def ResUNet(img_size):
#     f = [16, 32, 64, 128, 256]
#     inputs = Input((img_size, img_size, 1))
    
#     ## Encoder
#     e0 = inputs
#     e1 = stem(e0, f[0])
#     e2 = residual_block(e1, f[1], strides=2)
#     e3 = residual_block(e2, f[2], strides=2)
#     e4 = residual_block(e3, f[3], strides=2)
#     e5 = residual_block(e4, f[4], strides=2)
    
#     ## Bridge
#     b0 = conv_block(e5, f[4], strides=1)
#     b1 = conv_block(b0, f[4], strides=1)
    
#     ## Decoder
#     u1 = upsample_concat_block(b1, e4)
#     d1 = residual_block(u1, f[4])
    
#     u2 = upsample_concat_block(d1, e3)
#     d2 = residual_block(u2, f[3])
    
#     u3 = upsample_concat_block(d2, e2)
#     d3 = residual_block(u3, f[2])
    
#     u4 = upsample_concat_block(d3, e1)
#     d4 = residual_block(u4, f[1])
    
#     outputs = tf.keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(d4)
#     model = tf.keras.models.Model(inputs, outputs)
#     return model


# adam = tf.keras.optimizers.Adam(lr = 0.01, epsilon = 0.1)


Unet

In [ ]:
...
#model = Unet()
# class LossHistory(Callback):
#     def __init__(self, epochs):
#         self.epochs = epochs
        
#     def on_train_begin(self, logs={}):
#         self.losses = []
#         self.acc_logs = []
#         self.history_logs = []
#         self.check = 0
#         self.lrr = 0.001
#         print('Training Start.....')

#     def on_train_end(self, logs={}):
#         average_acc = sum(self.acc_logs)/len(self.acc_logs)
#         print("Average accury over {0} epochs is {1}".format(len(self.acc_logs),average_acc))
#         print("Training Done!")
        
#     def on_train_batch_begin(self, batch, logs=None):
#         pass

#     def on_train_batch_end(self, batch, logs=None):
#         pass
    
#     def on_epoch_begin(self, epoch,logs=None):
#         #print(K.get_value(self.model.optimizer.lr))
#         pass

#     def on_epoch_end(self, epoch, logs=None):
#         if(len(self.history_logs) >= 2):
#             curren_acc = round(float(logs.get('acc')),3)
#             last_acc = round(float(self.losses[-1]),3)
#             #print(" current is {0} and last is {1}".format(curren_acc,last_acc))
#             if(curren_acc == last_acc):
#                 self.check += 1
#             elif(curren_acc > last_acc):
#                 self.check = 0
#         #print(" {0} many bad attemps".format(history_callback.check))
#         self.losses.append(logs.get('acc'))
#         self.acc_logs.append(logs.get('acc'))
#         self.history_logs.append(len(self.losses))
#         self.lrr = step_decay(self.lrr)
#         self.model.optimizer.lr = self.lrr
        
# history_callback=LossHistory(50)
# def step_decay(learningRate):
#     if history_callback.check >= 5:
#         lrate=learningRate * 0.1
#         momentum=0.8
#         decay_rate=2e-6
#         print(" learning rate dropped to ",lrate)
#         return lrate
#     else:
#         return 0.001
# lrate=LearningRateScheduler(step_decay)           

# model.compile(optimizer=opt,loss="binary_crossentropy",metrics=["acc"])
# model.fit_generator(myGene,steps_per_epoch=2000,epochs=5,callbacks=[history_callback])#,validation_steps=valid_steps)


In [ ]:
model = unet()
#model = ResUNet(512)

model.compile(optimizer = Adam(lr = 1e-4), loss = bce_dice_loss, metrics = ['accuracy', dice_coef])

model_checkpoint = keras.callbacks.ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
print('number of train ',len(trainImage))
print('number of valid ',len(validImage))
trainStep = len(trainImage)//batch_size
validStep = len(validImage)//batch_size
history = model.fit_generator(myGene,steps_per_epoch=trainStep,epochs=80,callbacks=[model_checkpoint],\
                   validation_data = myValid, validation_steps = validStep)

In [ ]:
# opt = keras.optimizers.Adam(lr=0.001)
# #model.compile(optimizer=opt, loss=bce_dice_loss, metrics=["acc"])#metrics=[dsc]
# model.compile(optimizer=opt, loss=bce_dice_loss,  metrics = ['accuracy', dice_coef])#metrics=[dsc]
# #model.summary()

# trainStep = len(trainImage)//batch_size
# validStep = len(validImage)//batch_size
# history = model.fit_generator(myGene,steps_per_epoch=trainStep*2,epochs=200,validation_data=myValid,validation_steps=validStep)

In [ ]:
def show_final_history(history):
    fig= plt.figure(figsize=(10,10))
    plt.title('Loss')
    plt.plot(history.epoch, history.history["loss"], label="Train loss")
    plt.plot(history.epoch, history.history["val_loss"], label="Validation loss")
    plt.legend()
    fig= plt.figure(figsize=(10,10))
    plt.title('Acc')
    plt.plot(history.epoch, history.history["acc"], label="Train acc")
    plt.plot(history.epoch, history.history["val_acc"], label="Validation acc")
    plt.legend()
    fig= plt.figure(figsize=(10,10))
    plt.title('Dice_coef')
    plt.plot(history.epoch, history.history["dice_coef"], label="Train acc")
    plt.plot(history.epoch, history.history["val_dice_coef"], label="Validation acc")
    plt.legend()
show_final_history(history)
print("Validation Accuracy: " + str(history.history['val_acc'][-1:]))

**4. test model and zip output**

In [ ]:
from zipfile import ZipFile
import cv2

directory = os.listdir("../input/images/")
directory.sort()
#create output folder
if not os.path.exists("./output"):
    os.makedirs("./output")

def testGenerator(test_path,target_size = (512,512)):
    for i in test_path:
        i = "../input/images/" + i
        img = cv2.imread(i,0)
        img = img / 255

        img = trans.resize(img,target_size)
        img = np.reshape(img,img.shape+(1,))
        img = np.reshape(img,(1,)+img.shape)
        yield img
        
testGene = testGenerator(directory)
result = model.predict_generator(testGene,steps = len(directory),verbose=1)

saveResult("./output",result)
myoutputDir = "./output"

with ZipFile("./testOut.zip","w") as zip:
    for file in os.listdir(myoutputDir):
        zip.write("./output/" + file)
        
print(list(zip.infolist()))

In [ ]:
result.shape
plt.figure(0)
plt.imshow(result[0,:,:,0],'gray')
plt.show